# Search Engine with Tools and Agents

In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper

In [3]:
## Used the inbuilt tool of Wikipedia from Langchain Tools
api_wrapper_wiki= WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [6]:
## Used the inbuilt tool of Arxiv from Langchain Tools
api_wrapper_arxiv= ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [ ]:
tools=[wiki,arxiv]

In [13]:
## Custom Tools 
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()

documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())


In [15]:
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002957EBF1A20>, search_kwargs={})

In [16]:
## Convert retriver to tool
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any info about Langsmith")

In [17]:
retriever_tool.name

'langsmith-search'

In [18]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\faroo\\OneDrive\\Desktop\\Search-Engine-Langchain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any info about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002951DF579A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at

In [19]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")
openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [20]:
## Prompt Template : 2nd way of using it-not recommended if want customization
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [21]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [24]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [27]:
agent_executor.invoke({"input":"Explain LangSmith. What is it used for, and how does it work in LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'What is LangSmith?'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application per

{'input': 'Explain LangSmith. What is it used for, and how does it work in LangChain?',
 'output': "Thank you for the tool call output. Based on the information provided, LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows developers to closely monitor and evaluate their applications, making it easier to ship quickly and with confidence.\n\nLangSmith provides several key features, including:\n\n1. Observability: LangSmith offers LLM-native observability, which allows developers to get meaningful insights from their applications. This includes tracing, dashboards, and alerts for metrics like RPS, error rates, and costs.\n2. Evals: LangSmith provides a set of tools for building and running high-quality evaluations, including off-the-shelf evaluators, analysis of results, and collection of human feedback.\n3. Prompt Engineering: LangSmith offers a set of tools designed to enable and facilitate prompt engineering, which is the process of

In [28]:
agent_executor.invoke({"input":"Explain ReAct. What is it used for, and how does it work in AI Automation?"})



> Entering new AgentExecutor chain...
ReAct is a conversational AI system that uses a combination of natural language processing (NLP) and machine learning to understand and respond to user queries. In the context of AI automation, ReAct is used to provide a helpful assistant that can engage with users and respond to their questions and requests.

ReAct works by analyzing user input and identifying the intent behind the query. It then uses this intent to determine which tool or function to call in order to provide a relevant response. The tools and functions that ReAct has access to are defined in a set of tool definitions, which specify the parameters and requirements for each tool.

When a user asks a question, ReAct uses its NLP capabilities to analyze the input and identify the intent behind the query. It then checks the tool definitions to determine which tool or function is most relevant to the query. If a tool is identified, ReAct will call the tool with the appropriate parame

{'input': 'Explain ReAct. What is it used for, and how does it work in AI Automation?',
 'output': 'ReAct is a conversational AI system that uses a combination of natural language processing (NLP) and machine learning to understand and respond to user queries. In the context of AI automation, ReAct is used to provide a helpful assistant that can engage with users and respond to their questions and requests.\n\nReAct works by analyzing user input and identifying the intent behind the query. It then uses this intent to determine which tool or function to call in order to provide a relevant response. The tools and functions that ReAct has access to are defined in a set of tool definitions, which specify the parameters and requirements for each tool.\n\nWhen a user asks a question, ReAct uses its NLP capabilities to analyze the input and identify the intent behind the query. It then checks the tool definitions to determine which tool or function is most relevant to the query. If a tool is 

In [29]:
agent_executor.invoke({"input":"Explain Deep Learning. What is it used for?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Deep Learning'}`


Page: Deep learning
Summary: Deep learning is a subset of machine learning that focuses on utilizing multilayered neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from bioloThank you for calling the "wikipedia" tool. Based on the result, I'll continue to provide a more detailed explanation of Deep Learning.

Deep Learning is a subset of Machine Learning that focuses on using multilayered neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neural networks, where neurons connect and interact with each other to process and transmit information.

Deep Learning is particularly useful for tasks that require modeling complex patterns and relationships in data, such as:

1. Image and speech recognition: Deep Learning algorithms can 

{'input': 'Explain Deep Learning. What is it used for?',
 'output': 'Thank you for calling the "wikipedia" tool. Based on the result, I\'ll continue to provide a more detailed explanation of Deep Learning.\n\nDeep Learning is a subset of Machine Learning that focuses on using multilayered neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neural networks, where neurons connect and interact with each other to process and transmit information.\n\nDeep Learning is particularly useful for tasks that require modeling complex patterns and relationships in data, such as:\n\n1. Image and speech recognition: Deep Learning algorithms can be trained to recognize objects and patterns in images and audio files.\n2. Natural Language Processing (NLP): Deep Learning can be used to analyze and generate human language, including text and speech.\n3. Time series forecasting: Deep Learning can be used to forecast fu

In [30]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Attention Is All You Need" has been discussed.

> Finished chain.


{'input': "What's the paper 1706.03762 about?",
 'output': 'The paper "Attention Is All You Need" has been discussed.'}